# Working with geospatial data
In this notebook, we'll use a few of tools we explored in earlier examples and introduce the powerful `geopandas` library for working with and visulaizing geospatial vector data.

## [Nice overview tutorial](https://geopandas.org/en/stable/getting_started/introduction.html)
## [Examples Gallery](https://geopandas.org/en/stable/gallery/index.html)

In [ ]:
import geopandas as gpd

## Loading shapefiles with `geopandas`

Let's read in a shapefile of the United States using `read_file()`

In [ ]:
usa = gpd.read_file('data/cb_2018_us_state_20m.shp')

### `geopandas` is a lot like `pandas`, except with a special **geometry** column

In [ ]:
usa.head()

## you can use `pandas` operations to access your geodataframe, for example `loc`

In [ ]:
wisconsin = usa.loc[usa.NAME=='Wisconsin']

## you can easily look at your your shapefile spatially with `plot()`

In [ ]:
wisconsin.plot();

## Coordiate reference information is key
### the `crs` attribute has important metadata about the [spatial reference system](https://en.wikipedia.org/wiki/Spatial_reference_system)

In [ ]:
wisconsin.crs

### Often, CRS information are defined by [EPSG codes](https://epsg.org/home.html). Most common projections have one!
EPSG code [4269](https://spatialreference.org/ref/epsg/nad83/) is NAD83

## Reprojection is easy, just use `to_crs`
We can reproject our Wisconsin geodataframe to UTM zone 16 N (epsg 26916)

In [ ]:
wisconsin = wisconsin.to_crs(26916)
wisconsin.plot();

### Now try France Lambert 93 (not recommended)

In [ ]:
wisconsin = wisconsin.to_crs(2154)
wisconsin.plot();

### Finally, reproject back to our original CRS

In [ ]:
wisconsin = wisconsin.to_crs(4269)
wisconsin.plot();

# Let's make our own shapefile

## Question: Where are the wells with the oldest groundwater level measurements in Wisconsin?

We can use the `what_sites()` funciton from dataretrieval to get site data as a pandas DataFrame

In [ ]:
from dataretrieval import nwis
import pandas as pd

In [ ]:
df, md = nwis.what_sites(stateCd='WI', outputDataTypeCd='gw')

df

## Let's narrow this down and look at the oldest sites
wells with data from before January 1st, 1900

Convert start and end dates to datetime dtype

In [ ]:
df.begin_date = pd.to_datetime(df.begin_date)
df.end_date	= pd.to_datetime(df.begin_date)

In [ ]:
oldest_sites = df.loc[df.begin_date<pd.to_datetime('1900-01-01')].copy()

### Compute age in years using begin date

In [ ]:
oldest_sites['jstart'] = [i.to_julian_date() for i in oldest_sites.begin_date]
today = pd.to_datetime('today').to_julian_date()

oldest_sites['age_yrs'] = (today - oldest_sites['jstart']) / 365.25
oldest_sites

### Drop duplicate wells

In [ ]:
oldest_sites = oldest_sites.drop_duplicates(subset=['site_no'])
oldest_sites

## Convert this pandas DataFrame to a geopandas GeoDataFrame

### specify the geometry using lat and lon data and `points_from_xy`

In [ ]:
gdf = gpd.GeoDataFrame(oldest_sites, 
                       geometry=gpd.points_from_xy(x=oldest_sites.dec_long_va, 
                                                   y=oldest_sites.dec_lat_va), 
                       crs=4269) # also need to specify CRS

## check out the data spatially using the very cool `.explore()`

In [ ]:
gdf[['age_yrs', 'site_no', 'station_nm', 'geometry']].explore(column='age_yrs')

[1866!](https://waterdata.usgs.gov/nwis/inventory?agency_code=USGS&site_no=434821090454501)

## Make our own map

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots()
wisconsin.plot(ax=ax, fc='None', ec='black')
gdf.plot(ax=ax, column='age_yrs', legend=True, legend_kwds={"label": "Age in years"})
plt.title('The oldest water level data in Wisconsin');

## Save this new shapefile using `to_file`

In [ ]:
gdf.to_file('data/oldest_gw_sites.shp')